In [16]:
import pandas as pd
import numpy as np
import json
import os
import re
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.pandas.cursor import PandasCursor

In [17]:
# Load client data
client = "cooptotal"
client_df = pd.read_excel(f"./{client}_input_data.xlsx", converters={"Imps": int, "Placement Id": int})
client_df["Viewability Rate %"] = client_df["Viewability Rate %"].str.replace(",", ".").astype(float)
client_placement_ids = list(client_df["Placement Id"].unique())

In [18]:
# Fetch impressions using PyAthena library
cursor = connect(s3_staging_dir="s3://aws-athena-query-results-094611745175-eu-west-1/",
                 region_name="eu-west-1", profile_name="atexprodadminsso", cursor_class=PandasCursor).cursor()

impressions_df = cursor.execute(f'''
select 
    "impression_model"."id", 
    "hostname", 
    "placement_ids", 
    "placement_ids_chosen", 
    "total_fixation_duration", 
    "channel", 
    "ad_technical_format", 
    "is_fixated", 
    "gaze_valid", 
    "is_iab_inview", 
    "exist_viewable_1_s_threshold_50", 
    "exist_viewable_2_s_threshold_50", 
    "impression_model"."part_month", 
    CONCAT(cast("ad_width_chosen" as VARCHAR), 'x', cast("ad_width_chosen" as VARCHAR)) as size,
    "brand_batch"."chosen_brand" as "chosen_brand"
from "prod_attentionpanel_com_eu_west_1"."impression_model"
join "prod_attentionpanel_com_eu_west_1"."brand_batch" on impression_model.id = brand_batch.id
where impression_model.part_year = '2023'
''').as_pandas()

In [19]:
def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    try:
        ret = placement_id['tag_id'][0]
    except:
        return
    ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/tmp/ipykernel_15161/3375560408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))


In [20]:
# Get local predictions and store them in a map
def _get_pred_map():
    pred_map = dict()
    _year = 2023
    _range1 = 1
    _range2 = 10
    for i in range(_range1, _range2):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../../predictions/{_year}/{i}'):
            for h in os.listdir(f'../../predictions/{_year}/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../../predictions/{_year}/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [33]:
# Filter impressions based on tier
### TIER 1 ###
# This is the most accurate data we have. This is "campaign data", meaning correct brand and timeframe
### TIER 2 ###
# We might not have enough campaign data (tier 1). Tier 2 data is brand data, for a longer period (roughly 6 months)
### TIER 3 ###
# This is the least accurate data for a campaign. It ignore brand/timeframe and only checks Placement ID. This is mostly used as a benchmark
tier = 1
filtered_impressions = impressions_df[impressions_df["pid"].isin(client_placement_ids)]
if tier == 1:
    filtered_impressions = filtered_impressions[filtered_impressions["part_month"] == "09"]
if tier < 3:
    filtered_impressions = filtered_impressions[filtered_impressions["chosen_brand"] == client.lower()]

In [34]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

filtered_impressions['final_fixation'] = filtered_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

In [35]:
filtered_impressions["true_is_iab_inview"] = filtered_impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

In [36]:
filtered_impressions["total_fixation_duration"] = filtered_impressions["total_fixation_duration"].astype("Int64")
# Normalise outliers for fixation duration (30 seconds)
filtered_impressions.loc[filtered_impressions["total_fixation_duration"] >= 30000, "total_fixation_duration"] = 30000

In [37]:
# Predictions do not include fixation duration. For predicted fixations, use the median value as the total_fixation_duration

# Calculate the median fixation duration for each hostname
median_fixation_durations = filtered_impressions.loc[filtered_impressions['is_fixated'] == True].groupby('hostname')['total_fixation_duration'].median()

# Update the total_fixation_duration based on the calculated medians
filtered_impressions['total_fixation_duration'] = np.where(
    (filtered_impressions['total_fixation_duration'] == 0) &
    (filtered_impressions['final_fixation'] == True) &
    (filtered_impressions['hostname'].isin(median_fixation_durations.index)),
    filtered_impressions['hostname'].map(median_fixation_durations),
    filtered_impressions['total_fixation_duration']
)

In [38]:
grouped_df = filtered_impressions.groupby(["pid", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "impressions": x["id"].count(),
            "fixations": x.loc[x["final_fixation"], "id"].count(),
            "inview": x.loc[x["true_is_iab_inview"], "id"].count(),
            "total_fixation_duration": x["total_fixation_duration"].sum(),
            "average_fixation_duration": x["total_fixation_duration"].mean()
        }
    )
)
grouped_df = grouped_df.reset_index()

In [39]:
grouped_client = client_df.groupby(["Placement Id"]).apply(
    lambda x: pd.Series(
        {
            "Imps": x["Imps"].sum(),
            "Viewability Rate %": np.average(x["Viewability Rate %"], weights=x["Imps"])
        }
    )
)
grouped_client = grouped_client.reset_index()

In [40]:
merged_df = grouped_df.merge(grouped_client, how="inner", left_on="pid", right_on="Placement Id")
merged_df = merged_df[["hostname", "Placement Id", "impressions", "fixations", "inview", "Imps", "Viewability Rate %", "average_fixation_duration", "total_fixation_duration"]]

In [41]:
merged_df["fixation_ratio"] = round(merged_df["fixations"] / merged_df["impressions"], 2)
merged_df["client_fixations_per_placement"] = np.floor(merged_df["fixation_ratio"] * merged_df["Imps"]).fillna(0).replace([np.inf, -np.inf], 0)
merged_df["client_total_fixation_duration_per_placement"] = np.floor(merged_df["average_fixation_duration"] * merged_df["client_fixations_per_placement"]).fillna(0).replace([np.inf, -np.inf], 0)

In [42]:
final_group = merged_df.groupby(["hostname"]).apply(
     lambda x: pd.Series(
         {
             "tobii_imps": x["impressions"].sum(),
             "client_imps": x["Imps"].sum(),
             "tobii_fixations": x["fixations"].sum(),
             "tobii_inview": x["inview"].sum(),
             "client_fixations_per_placement": x["client_fixations_per_placement"].sum(),
             "client_viewability_rate": np.average(x["Viewability Rate %"], weights=x["Imps"]),
             "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 2),             
             "average_fixation_duration_per_placement": np.floor(x["client_total_fixation_duration_per_placement"].sum() / x["client_fixations_per_placement"].sum()), 
             "total_fixation_duration": x["total_fixation_duration"].sum(),
             "client_total_fixation_duration_per_placement": x["client_total_fixation_duration_per_placement"].sum()
         }
     )
)
final_group = final_group.reset_index()
final_group["tobii_imps"] = final_group["tobii_imps"].astype("Int64")
final_group["client_imps"] = final_group["client_imps"].astype("Int64")
final_group["tobii_fixations"] = final_group["tobii_fixations"].astype("Int64")
final_group["tobii_inview"] = final_group["tobii_inview"].astype("Int64")
final_group["tobii_inview/impression_ratio"] = round(final_group["tobii_inview"] / final_group["tobii_imps"] * 100, 2)
final_group["tobii_fixation/impression_ratio"] = round(final_group["tobii_fixations"] / final_group["tobii_imps"] * 100, 2)

In [43]:
final_group["client_inview"] = np.round(final_group["client_imps"] * final_group["client_viewability_rate"]).astype("Int64")
final_group["client_fixations"] = np.round((final_group["tobii_fixations"] / final_group["tobii_imps"]) * final_group["client_imps"]).astype("Int64")
final_group["client_fix/inview_ratio"] = round(final_group["client_fixations"] / final_group["client_inview"] * 100, 2)
final_group["client_total_fixation_duration"] = final_group["client_fixations"] * final_group["average_fixation_duration"]

def get_sample_size(num_impressions):
    if num_impressions <= 99:
        return "low"
    elif num_impressions >= 100 and num_impressions <= 199:
        return "medium"
    else:
        return "high"

final_group["sample_size"] = final_group["tobii_imps"].apply(lambda x: get_sample_size(x))

In [44]:
final_group

,hostname,tobii_imps,client_imps,tobii_fixations,tobii_inview,client_fixations_per_placement,client_viewability_rate,average_fixation_duration,average_fixation_duration_per_placement,total_fixation_duration,client_total_fixation_duration_per_placement,tobii_inview/impression_ratio,tobii_fixation/impression_ratio,client_inview,client_fixations,client_fix/inview_ratio,client_total_fixation_duration,sample_size
0,aftonbladet.se,424,1407515,157,240,525551.0,0.758825,1107.87,464.0,173935.0,243857215.0,56.6,37.03,1068057,521179,48.8,577398578.73,high
1,malservice.aftonbladet.se,7,11,1,1,1.0,0.545455,0.00,0.0,0.0,0.0,14.29,14.29,6,2,33.33,0.0,low
2,svd.se,1,1616,1,1,1616.0,0.687920,1056.00,1056.0,1056.0,1706496.0,100.0,100.0,1112,1616,145.32,1706496.0,low


In [32]:
result_df = final_group[["hostname", "tobii_imps", "client_imps", "tobii_fixations", "client_fixations", "tobii_inview", "client_inview", "client_fix/inview_ratio", "tobii_inview/impression_ratio", "client_viewability_rate", "tobii_fixation/impression_ratio", "average_fixation_duration", "total_fixation_duration", "client_total_fixation_duration", "sample_size"]]
result_df
#result_df.to_excel(f"./{client.lower()}_tier_{tier}_example_report.xlsx", index=False)

,hostname,tobii_imps,client_imps,tobii_fixations,client_fixations,tobii_inview,client_inview,client_fix/inview_ratio,tobii_inview/impression_ratio,client_viewability_rate,tobii_fixation/impression_ratio,average_fixation_duration,total_fixation_duration,client_total_fixation_duration,sample_size
0,aftonbladet.se,424,1407515,157,521179,240,1068057,48.8,56.6,0.758825,37.03,1107.87,173935.0,577398578.73,high
1,malservice.aftonbladet.se,7,11,1,2,1,6,33.33,14.29,0.545455,14.29,0.00,0.0,0.0,low
2,svd.se,1,1616,1,1616,1,1112,145.32,100.0,0.687920,100.0,1056.00,1056.0,1706496.0,low


In [43]:
# Load tiered data
df_1 = pd.read_excel(f"./{client.lower()}_tier_1_example_report.xlsx")
df_1["tier"] = 1
df_2 = pd.read_excel(f"./{client.lower()}_tier_2_example_report.xlsx")
df_2["tier"] = 2
df_3 = pd.read_excel(f"./{client.lower()}_tier_3_example_report.xlsx")
df_3["tier"] = 3
all_df = [df_1, df_2, df_3]
combined_client_df = pd.concat(all_df)
columns = list(df_1.columns)

In [44]:
def get_data_based_on_tier(client_dfs):
    new_df = pd.DataFrame(columns=columns)
    for i, df in enumerate(client_dfs):
        for index, row in df.iterrows():
            if row.tier == 1:
                if row.sample_size in ("high", "medium"):
                    new_df = new_df.append(row)
            elif row.tier == 2:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    if row.sample_size == "high":
                        # Degrade sample_size if lower tier
                        row.sample_size = "medium"
                        new_df = new_df.append(row)
                    elif row.sample_size == "medium":
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            np.floor(row["client_fixations"] * 0.25 + new_row["client_fixations"].sum() * 0.75)
                        )
                        new_df = new_df.append(new_row)
                    else:
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            np.floor(row["client_fixations"] * 0.15 + new_row["client_fixations"].sum() * 0.85)
                        )
                        new_df = new_df.append(new_row)
            elif row.tier == 3:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    row["sample_size"] = "low"
                    new_df = new_df.append(row)
    return new_df

final_client_df = get_data_based_on_tier(all_df)

/tmp/ipykernel_8030/966106096.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
/tmp/ipykernel_8030/966106096.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.sample_size = "medium"
/tmp/ipykernel_8030/966106096.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row["client_fixations"] = (
/tmp/ipykernel_8030/966106096.py:27: FutureWarning: The frame.append method is depre

In [45]:
def do_math(row, arg1, arg2):
    try:
        return round(row[f"{arg1}"] / row[f"{arg2}"] * 100, 2)
    except ZeroDivisionError:
        return 0

final_client_df["client_Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_inview", "client_imps"), axis=1)
final_client_df["client_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_client_df["tobii_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_client_df["client_fixation/impression_ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_client_df["Client Total Fixation Duration"] = final_client_df["average_fixation_duration"] * final_client_df["client_fixations"]
final_client_df = final_client_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "Client Total Fixation Duration", "total_fixation_duration", "average_fixation_duration", "sample_size", "tier"]]
final_client_df = final_client_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        "client_Inview Ratio": "Inview Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "average_fixation_duration": "Average Fixation Duration",
        "total_fixation_duration": "Total Fixation Duration",
        "sample_size": "Sample Size"
    }
)
final_client_df

,hostname,Impressions,Fixations,tobii_fixation/impression_ratio,Inviews,Inview Ratio,Fixation/Inview Ratio,Fixation/Tobii Inview Ratio,tobii_inview/impression_ratio,Client Total Fixation Duration,Total Fixation Duration,Average Fixation Duration,Sample Size,tier
0,aftonbladet.se,1387320,572270,41.25,1053056,75.91,54.34,71.74,57.50,424578558.4,73450,741.92,high,1
2,bloggar.aftonbladet.se,155594,78661.0,54.51,110212,70.83,71.37,100.72,54.12,267160287.35,472093.0,3396.35,medium,3
3,klart.se,7023,1882.0,30.02,4510,64.22,41.73,53.96,55.63,3731591.96,729662.0,1982.78,medium,3
6,svd.se,98759,42847.0,50.72,79722,80.72,53.75,78.38,64.70,360401970.39,42519482.0,8411.37,medium,3
1,blocket.se,4058,1691,41.67,3342,82.36,50.60,52.63,79.17,0.0,0.0,0.00,low,3
4,malservice.aftonbladet.se,11,2,20.37,6,54.55,33.33,61.11,33.33,1286.46,7075.5,643.23,low,3
5,skonhetsredaktorerna.se,86770,86770,100.00,66274,76.38,130.93,100.00,100.00,0.0,0.0,0.00,low,3
7,tramsfrans.aftonbladet.se,86770,0,0.00,66274,76.38,0.00,0.00,100.00,NaN,0.0,NaN,low,3
8,tv.nu,17359,4676,26.94,12685,73.07,36.86,45.72,58.92,10801840.56,505903.0,2310.06,low,3


In [46]:
final_client_df = final_client_df[["hostname", "Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration", "Client Total Fixation Duration", "Sample Size"]]
final_client_df = final_client_df.sort_values(by=["hostname"])

In [47]:
comments = ["Tier 1", "Tier 2", "Tier 3"]

with pd.ExcelWriter(f"./{client}_results_internal.xlsx") as writer:
    rows = 0
    spaces = 0
    for n, df in enumerate(all_df):
        pd.Series(comments[n]).to_excel(writer, sheet_name=client.title(), index=False, header=False, startrow=rows + spaces)
        df.to_excel(writer, client.title(), index=False, startrow=1 + rows + spaces)
        rows += len(df) + 2
        spaces += 2

with pd.ExcelWriter(f"./{client}_results.xlsx") as writer:
    for df in [final_client_df]:
        df.to_excel(writer, sheet_name=client.title(), index=False)